In [1]:
import os
import pandas as pd
from tqdm import tqdm
from tqdm import trange
from glob import glob
workdir = ''
rna_df = pd.read_csv(r'E:\TMC\PRISM_point_typing\dataset\20231218_PRISM30_DGA_1\endo_labeled.csv')[['Y','X','Gene']]
rna_df['Y'] = rna_df['Y'].astype(int)
rna_df['X'] = rna_df['X'].astype(int)
genes = set(rna_df['Gene'])
rna_df.head()

,Y,X,Gene
0,6399,1249,PRISM_7
1,11581,9484,PRISM_7
2,2393,8189,PRISM_7
3,2862,8712,PRISM_7
4,10829,6472,PRISM_7


In [2]:
from draw_spots import crop_df
from skimage.io import imread
from skimage.io import imsave
import numpy as np
from tqdm import tqdm
from tqdm import trange

im = np.zeros((13575, 13548),dtype=np.uint16)

def plot_density_downsample(df,filename,fac=50):
    y = (im.shape[0] // fac) + 1
    x = (im.shape[1] // fac) + 1
    coordinates = df[['Y','X']].to_numpy()
    canvas = np.zeros((y*fac,x*fac),dtype=np.uint16)
    canvas[coordinates[:,0],coordinates[:,1]] = 1
    canvas_down = canvas.reshape(y,fac,x,fac).sum(-1).sum(1)
    imsave(filename,canvas_down.astype(np.uint16),check_contrast=False)

try:
    os.mkdir('./density_31')
except FileExistsError:
    pass

for gene in tqdm(genes):
    if glob(os.path.join('./density_31',f'{gene}.tif')):
        continue
    else:
        df = rna_df[rna_df['Gene']==gene]
        plot_density_downsample(df,f'./density_31/{gene}.tif')


100%|██████████| 31/31 [00:06<00:00,  4.63it/s]


In [12]:
from draw_spots import crop_df
from skimage.io import imread
from skimage.io import imsave
import numpy as np
from tqdm import tqdm
from tqdm import trange

im = np.zeros((46781,33466),dtype=np.uint16)

def plot_density_downsample(df,filename,fac=100):
    y = (im.shape[0] // fac) + 1
    x = (im.shape[1] // fac) + 1
    coordinates = df[['Y','X']].to_numpy()
    canvas = np.zeros((y*fac,x*fac),dtype=np.uint16)
    canvas[coordinates[:,0],coordinates[:,1]] = 1
    canvas_down = canvas.reshape(y,100,x,100).sum(-1).sum(1)
    imsave(filename,canvas_down.astype(np.uint16),check_contrast=False)

try:
    os.mkdir('./density_RMdup')
except FileExistsError:
    pass

for gene in tqdm(genes):
    if glob(os.path.join('./density_RMdup',f'{gene}.tif')):
        continue
    else:
        df = rna_df[rna_df['Gene']==gene]
        plot_density_downsample(df,f'./density_RMdup/{gene}.tif')

100%|██████████| 11/11 [00:19<00:00,  1.79s/it]


In [34]:
from itertools import combinations
import yaml
import ast
def hamming(x,y):
    hamm = 0
    for a,b in zip(x,y):
        if a!=b:
            hamm += 1
    return hamm
def overlap(seq_x,seq_y):
    output = ''
    for x,y in zip(seq_x,seq_y):
        if x == y:
            output += x
        elif 'G' not in x+y:
            output += 'A'
        else:
            if 'C' in x+y:
                output += 'C'
            elif 'T' in x+y:
                output += 'T'
    return output
df = pd.read_csv('mapped_tophat_raw.csv')[['Y','X','Gene']]
df['Gene'] = df['Gene'].apply(lambda x: ast.literal_eval(x))
df['Gene'] = df['Gene'].apply(lambda x: ','.join(x))
# df.columns = ['Y','X','Name']
# df['Gene'] = df['Name'].apply(lambda x: x.split('-')[0])
# rna_df['Gene'] = rna_df['Gene'].apply(lambda x: x.split('-')[0])
ref_df = pd.read_excel('0811_gene_49_full.xlsx')[['Gene','Inner Index','Barcode']]
ref_df['Inner Index'] = ref_df['Inner Index'].astype(str)
ref_df['Name'] = ref_df[['Gene','Inner Index']].agg('-'.join, axis=1)
df_gene_group = ref_df.groupby('Gene')
genes_dict = {}
for group in df_gene_group.groups:
    temp_df = df_gene_group.get_group(group)
    pairs = tuple(combinations(temp_df.index,2))
    gene_dict = {}
    for pair in pairs:
        name_x = temp_df.loc[pair[0],'Name']
        name_y = temp_df.loc[pair[1],'Name']
        seq_x = temp_df.loc[pair[0],'Barcode']
        seq_y = temp_df.loc[pair[1],'Barcode']
        seq_xy = overlap(seq_x,seq_y)
        count_dict = {}
        count_dict[seq_x] = len(df[df['Gene']==name_x])
        count_dict[seq_y] = len(df[df['Gene']==name_y])
        count_dict[seq_xy] = len(df[df['Gene']==(f'{name_x},{name_y}' or f'{name_y},{name_x}')])
        gene_dict[f'{name_x},{name_y}'] = count_dict
    genes_dict[group] = gene_dict
with open('overlap_meta.txt','w') as f:
    yaml.dump(genes_dict,f,default_flow_style=False)

In [3]:
print(len(rna_df))

24436523
